In [31]:
import pandas as pd
import numpy as np
import warnings

pd.set_option("mode.copy_on_write", True)

In [32]:
poverty_data=pd.read_csv("USDA_data/Poverty.csv")
# columns name : ['FIPS*', 'Name', 'RUC Code', 'All people in poverty (2021) Percent', 'Children ages 0-17 in poverty (2021) Percent']
poverty_data.rename(columns={'FIPS*': 'FIPS'}, inplace=True)
poverty_data["YR"]="2021" # add a column as Year
poverty_data.head(5)

,FIPS,Name,RUC Code,All people in poverty (2021) Percent,Children ages 0-17 in poverty (2021) Percent,YR
0,23000,Maine,NaN,11.2,13.8,2021
1,23001,Androscoggin,3.0,14.0,19.1,2021
2,23003,Aroostook,7.0,14.9,18.9,2021
3,23005,Cumberland,2.0,7.7,8.7,2021
4,23007,Franklin,8.0,11.7,15.7,2021


In [33]:
education_CompletingCollege=pd.read_csv("USDA_data/Education(CompletingCollege).csv")
education_CompletingCollege.rename(columns={'2023 Rural-urban Continuum Code*': 'RUC Code'}, inplace=True)

education_CompletingCollege_melt = education_CompletingCollege.melt(
    id_vars=['FIPS', 'Name','RUC Code'],
    var_name='YR',
    value_name='Completing College'
)

education_CompletingHighSchool_only=pd.read_csv("USDA_data/Education(CompletingHighSchool_only).csv")
education_CompletingHighSchool_only.rename(columns={'2023 Rural-urban Continuum Code*': 'RUC Code'}, inplace=True)

education_CompletingHighSchool_only_melt  = education_CompletingHighSchool_only.melt(
    id_vars=['FIPS', 'Name','RUC Code'],
    var_name='YR',
    value_name='Completing High School Only'
)


education_NotCompletingHighSchool=pd.read_csv("USDA_data/Education(NotCompletingHighSchool).csv")
education_NotCompletingHighSchool.rename(columns={'2023 Rural-urban Continuum Code*': 'RUC Code'}, inplace=True)
education_NotCompletingHighSchool_melt = education_NotCompletingHighSchool.melt(
    id_vars=['FIPS', 'Name','RUC Code'],
    var_name='YR',
    value_name='Not Completing High School'
)


In [34]:
education_CompletingHighSchool_only_2 = education_CompletingHighSchool_only_melt.drop(columns=['Name', 'RUC Code'])
education_NotCompletingHighSchool_2 = education_NotCompletingHighSchool_melt.drop(columns=['Name', 'RUC Code'])
# in case RUC Code and Name different expression

education_data = (
    education_CompletingCollege_melt
    .merge(education_CompletingHighSchool_only_2, on=['FIPS', 'YR'], how='inner')  # Inner join on FIPS and YR
    .merge(education_NotCompletingHighSchool_2, on=['FIPS', 'YR'], how='inner')   # Another inner join
    
)

education_data.head(5)

# check inner
# education_data = (
#     education_CompletingCollege_melt
#     .merge(education_CompletingHighSchool_only_2, on=['FIPS', 'YR'], how='outer', indicator=True)
#     .rename(columns={'_merge': '_merge_college_highschool'})  # Rename the first `_merge` column
#     .merge(education_NotCompletingHighSchool_2, on=['FIPS', 'YR'], how='outer', indicator=True)
#     .rename(columns={'_merge': '_merge_final'})  # Rename the final `_merge` column
# )

# # Analyze the final merge summary
# merge_summary = education_data['_merge_final'].value_counts()

# # Display the merge summary
# print("Summary of the final merge:")
# print(merge_summary)

,FIPS,Name,RUC Code,YR,Completing College,Completing High School Only,Not Completing High School
0,12000,Florida,NaN,1970,10.3%,30.7%,47.4%
1,12001,"Alachua, FL",2.0,1970,23.1%,24.5%,40.2%
2,12003,"Baker, FL",1.0,1970,3.6%,26.5%,65.1%
3,12005,"Bay, FL",3.0,1970,9.2%,31.5%,49.0%
4,12007,"Bradford, FL",6.0,1970,4.5%,25.7%,63.5%


In [35]:
# unemployment_income=pd.read_csv("USDA_data/UnemploymentRate(%).csv")
# unemployment_income.columns = unemployment_income.iloc[0]
# unemployment_income = unemployment_income[2:].reset_index(drop=True)
# unemployment_income = unemployment_income.iloc[:, :-1]
# unemployment_income.rename(columns={'FIPS ': 'FIPS'}, inplace=True)
# unemployment_income.head(5)


In [36]:
# income_data = unemployment_income[['FIPS', 'Name', 'Median Household Income (2021)']]
# income_data["YR"]="2021" # add a column as Year
# income_data.head(5)

In [37]:
# unemployment_data=unemployment_income.iloc[:, :-1]
# unemployment_data.head(5)
# unemployment_data = unemployment_data.melt(
#     id_vars=['FIPS', 'Name'],
#     var_name='YR',
#     value_name='Unemployment Rate (%)'
# )
# unemployment_data['YR'] = unemployment_data['YR'].astype(int)
# unemployment_data.head(5)

In [38]:
unemployment_all=pd.read_csv("USDA_data/Unemployment.csv")
unemployment_all=unemployment_all[unemployment_all["State"].isin(["FL", "WA", "OR", "GA", "OK", "AL", "CO", "ME"])]
unemployment_all_melt = unemployment_all.melt(
    id_vars=['FIPS_Code', 'State', 'Area_Name'],
    var_name='Metric',
    value_name='value'
)
unemployment_all_melt['YR'] = unemployment_all_melt['Metric'].str[-4:]
unemployment_all_melt['Metric'] = unemployment_all_melt['Metric'].str[:-5]

unemployment_all_pivot = unemployment_all_melt.pivot(
    index=['FIPS_Code', 'State', 'Area_Name', 'YR'],  # Identifiers for rows
    columns='Metric',                              # Column headers
    values='value'                                 # Values to populate
)

unemployment_all_pivot.reset_index(inplace=True)
unemployment_all_data = unemployment_all_pivot[["FIPS_Code", "State", "Area_Name", "YR", "Civilian_labor_force", "Unemployment_rate","Med_HH_Income_Percent_of_State_Total","Median_Household_Income"]]
unemployment_all_data.rename(columns={'FIPS_Code': 'FIPS'}, inplace=True)
unemployment_all_data.rename(columns={'Area_Name': 'Name'}, inplace=True)
unemployment_all_data

Metric,FIPS,State,Name,YR,Civilian_labor_force,Unemployment_rate,Med_HH_Income_Percent_of_State_Total,Median_Household_Income
0,1000,AL,Alabama,2000,"2,147,173",4.6,NaN,NaN
1,1000,AL,Alabama,2001,"2,128,027",5.2,NaN,NaN
2,1000,AL,Alabama,2002,"2,112,621",5.9,NaN,NaN
3,1000,AL,Alabama,2003,"2,128,668",6.0,NaN,NaN
4,1000,AL,Alabama,2004,"2,138,306",5.6,NaN,NaN
...,...,...,...,...,...,...,...,...
12254,53077,WA,"Yakima County, WA",2018,"129,654",6.3,NaN,NaN
12255,53077,WA,"Yakima County, WA",2019,"133,308",7.1,NaN,NaN
12256,53077,WA,"Yakima County, WA",2020,"132,309",9.5,NaN,NaN
12257,53077,WA,"Yakima County, WA",2021,"130,342",7.0,71.4,"60,058"


In [39]:
population=pd.read_csv("USDA_data/Population.csv")
population_filted=population[population["State"].isin(["FL", "WA", "OR", "GA", "OK", "AL", "CO", "ME"])]
population_filted_melt = population_filted.melt(
    id_vars=['FIPStxt', 'State', 'Area_Name'],
    var_name='Metric',
    value_name='value'
)
population_filted_melt['YR'] = population_filted_melt['Metric'].str[-4:]
population_filted_melt['Metric'] = population_filted_melt['Metric'].str[:-5]
population_filted_pivot = population_filted_melt.pivot(
    index=['FIPStxt', 'State', 'Area_Name', 'YR'],  # Identifiers for rows
    columns='Metric',                              # Column headers
    values='value'                                 # Values to populate
)

population_filted_pivot.reset_index(inplace=True)
population_filted_pivot.columns

Index(['FIPStxt', 'State', 'Area_Name', 'YR', 'BIRTHS', 'CENSUS_202', 'DEATHS',
       'DOMESTIC_MIG', 'ESTIMATES_BASE', 'Economic_typology', 'GQ_ESTIMATES',
       'GQ_ESTIMATES_BASE', 'INTERNATIONAL_MIG', 'NATURAL_CHG', 'NET_MIG',
       'N_POP_CHG', 'POP_ESTIMATE', 'RESIDUAL', 'R_BIRTH', 'R_DEATH',
       'R_DOMESTIC_MIG', 'R_INTERNATIONAL_MIG', 'R_NATURAL_CHG', 'R_NET_MIG',
       'Rural_Urban_Continuum_Code', 'Urban_Influence'],
      dtype='object', name='Metric')

CENSUS_202: Could indicate population counts or census data for 2020.  
ESTIMATES_BASE: Likely represents the baseline population estimates.  
POP_ESTIMATE: Most likely the estimated total population.  
N_POP_CHG: Represents net population change.  
R_NATURAL_CHG: Rate of natural population change (births minus deaths).  
R_NET_MIG: Rate of net migration, impacting population.  

In [40]:
population_data = population_filted_pivot[["FIPStxt", "State", "Area_Name", "YR", "POP_ESTIMATE", "N_POP_CHG", "R_NATURAL_CHG"]]
population_data.rename(columns={'FIPStxt': 'FIPS'}, inplace=True)
population_data.rename(columns={'Area_Name': 'Name'}, inplace=True)
population_data.rename(columns={'POP_ESTIMATE': 'Estimated Population'}, inplace=True)
population_data.rename(columns={'N_POP_CHG': 'Net Population Change'}, inplace=True)
population_data.rename(columns={'R_NATURAL_CHG': 'Rate of Poulation Change(birth minus deaths)'}, inplace=True)
population_data = population_data.dropna(subset=["Estimated Population", "Net Population Change", "Rate of Poulation Change(birth minus deaths)"])
population_data.head(5)


Metric,FIPS,State,Name,YR,Estimated Population,Net Population Change,Rate of Poulation Change(birth minus deaths)
3,1000,AL,Alabama,2021,"5,050,380","18,516",-2.4
4,1000,AL,Alabama,2022,"5,073,903","23,523",-1.8
5,1000,AL,Alabama,2023,"5,108,468","34,565",-0.3
10,1001,AL,Autauga County,2021,"59,203",288,-0.2
11,1001,AL,Autauga County,2022,"59,726",523,0.3


# Merge all dataset

In [41]:
#check data type, prepare for merging

# print(education_data.dtypes)
# print(poverty_data.dtypes)
# print(income_data.dtypes)
# print(unemployment_data.dtypes)
# print(population_data.dtypes)
# print(unemployment_all_data.dtypes)
education_data['YR'] = education_data['YR'].astype('str')
poverty_data['YR'] = poverty_data['YR'].astype('str')
# income_data['FIPS'] = income_data['FIPS'].astype('int64')
# income_data['YR'] = income_data['YR'].astype('str')
# unemployment_data['YR'] = unemployment_data['YR'].astype('str')
# unemployment_data['FIPS'] = unemployment_data['FIPS'].astype('int64')
population_data['YR'] = population_data['YR'].astype('str')
unemployment_all_data['YR'] = unemployment_all_data['YR'].astype('str')


In [42]:
unemployment_all_data.head(5)

Metric,FIPS,State,Name,YR,Civilian_labor_force,Unemployment_rate,Med_HH_Income_Percent_of_State_Total,Median_Household_Income
0,1000,AL,Alabama,2000,"2,147,173",4.6,NaN,NaN
1,1000,AL,Alabama,2001,"2,128,027",5.2,NaN,NaN
2,1000,AL,Alabama,2002,"2,112,621",5.9,NaN,NaN
3,1000,AL,Alabama,2003,"2,128,668",6.0,NaN,NaN
4,1000,AL,Alabama,2004,"2,138,306",5.6,NaN,NaN


In [43]:
poverty_data=poverty_data.drop(columns=['Name', 'RUC Code'])
# income_data=income_data.drop(columns='Name')
# unemployment_data=unemployment_data.drop(columns='Name')
population_data=population_data.drop(columns=['Name', 'State'])
unemployment_all_data=unemployment_all_data.drop(columns=['Name', 'State'])

In [44]:

# Merge all the datasets on 'FIPS' and 'YR'
merged_data = (
    education_data
    .merge(poverty_data, on=['FIPS', 'YR'], how='outer')  # Adjust 'how' if necessary
    # .merge(income_data, on=['FIPS', 'YR'], how='outer')
    # .merge(unemployment_data, on=['FIPS', 'YR'], how='outer')
    .merge(unemployment_all_data,on=['FIPS', 'YR'], how='outer')
    .merge(population_data, on=['FIPS', 'YR'], how='outer')
)
merged_data.head(5)

,FIPS,Name,RUC Code,YR,Completing College,Completing High School Only,Not Completing High School,All people in poverty (2021) Percent,Children ages 0-17 in poverty (2021) Percent,Civilian_labor_force,Unemployment_rate,Med_HH_Income_Percent_of_State_Total,Median_Household_Income,Estimated Population,Net Population Change,Rate of Poulation Change(birth minus deaths)
0,1000,Alabama,NaN,1970,7.8%,25.9%,58.7%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000,Alabama,NaN,1980,12.2%,31.8%,43.5%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000,Alabama,NaN,1990,15.7%,29.4%,33.1%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000,Alabama,NaN,2000,19.0%,30.4%,24.7%,NaN,NaN,"2,147,173",4.6,NaN,NaN,NaN,NaN,NaN
4,1000,NaN,NaN,2001,NaN,NaN,NaN,NaN,NaN,"2,128,027",5.2,NaN,NaN,NaN,NaN,NaN


In [45]:
merged_data.to_parquet("data/USDA_education_poverty_unemployment_income.parquet", index=False)
